# AB Testing

## Import Library

In [1]:
import pandas as pd
import numpy as np

In [4]:
ab_df = pd.read_csv(r'ab_data.csv')
ab_df

FileNotFoundError: [Errno 2] No such file or directory: 'ab_data.csv'

In [107]:
ab_df = ab_df[['user_id', 'timestamp', 'group', 'landing_page', 'converted']]

In [108]:
len(ab_df)

294478

In [111]:
len(ab_df.user_id.unique())

290584

In [116]:
counter = ab_df['user_id'].value_counts()
(counter > 1).value_counts()

count
True     286690
False      3894
Name: count, dtype: int64

In [102]:
valid_users = pd.DataFrame(counter[counter == 1].index, columns=['user_id'])
ab_df = ab_df.merge(valid_users, on=['user_id'])
ab_df

In [92]:
count_df = ab_df.groupby('user_id').count().reset_index().sort_values(by='group', ascending=False)
count_df


,user_id,timestamp,group,landing_page,converted
91655,729669,2,2,2,2
179531,825266,2,2,2,2
30345,663033,2,2,2,2
41348,674959,2,2,2,2
225511,875210,2,2,2,2
...,...,...,...,...,...
97535,736032,1,1,1,1
97536,736033,1,1,1,1
97537,736034,1,1,1,1
97538,736035,1,1,1,1


In [104]:
count_df[count_df['group']==1]

,user_id,timestamp,group,landing_page,converted
191971,838798,1,1,1,1
194744,841817,1,1,1,1
191967,838794,1,1,1,1
194249,841281,1,1,1,1
194241,841273,1,1,1,1
...,...,...,...,...,...
97535,736032,1,1,1,1
97536,736033,1,1,1,1
97537,736034,1,1,1,1
97538,736035,1,1,1,1


In [93]:
len(count_df[count_df['group']==2])/len(count_df)

0.013400600170690747

In [94]:
len(count_df)

290584

In [117]:
ab_df = ab_df.drop_duplicates(subset=['user_id'],keep=False)
ab_df

,user_id,timestamp,group,landing_page,converted
0,851104,1/21/17 10:11 PM,control,old_page,0
1,804228,1/12/17 8:01 AM,control,old_page,0
2,661590,1/11/17 4:55 PM,treatment,new_page,0
3,853541,1/8/17 6:28 PM,treatment,new_page,0
4,864975,1/21/17 1:52 AM,control,old_page,1
...,...,...,...,...,...
294473,751197,1/3/17 10:28 PM,control,old_page,0
294474,945152,1/12/17 12:51 AM,control,old_page,0
294475,734608,1/22/17 11:45 AM,control,old_page,0
294476,697314,1/15/17 1:20 AM,control,old_page,0


In [119]:
ab_df = ab_df[~(ab_df['group']=='control') | ~(ab_df['landing_page']=='new_page')]
ab_df = ab_df[~(ab_df['group']=='treatment') | ~(ab_df['landing_page']=='old_page')]
ab_df

,user_id,timestamp,group,landing_page,converted
0,851104,1/21/17 10:11 PM,control,old_page,0
1,804228,1/12/17 8:01 AM,control,old_page,0
2,661590,1/11/17 4:55 PM,treatment,new_page,0
3,853541,1/8/17 6:28 PM,treatment,new_page,0
4,864975,1/21/17 1:52 AM,control,old_page,1
...,...,...,...,...,...
294473,751197,1/3/17 10:28 PM,control,old_page,0
294474,945152,1/12/17 12:51 AM,control,old_page,0
294475,734608,1/22/17 11:45 AM,control,old_page,0
294476,697314,1/15/17 1:20 AM,control,old_page,0


In [120]:
ab_df.groupby('group').count()

,user_id,timestamp,landing_page,converted
group,,,,
control,143293,143293,143293,143293
treatment,143397,143397,143397,143397


In [121]:
old_conversion = len(ab_df[(ab_df['group']=='control') & (ab_df['converted']==1)])/len(ab_df[(ab_df['group']=='control')])
old_conversion

0.12017335110577627

In [122]:
new_conversion = len(ab_df[(ab_df['group']=='treatment') & (ab_df['converted']==1)])/len(ab_df[(ab_df['group']=='treatment')])
new_conversion

0.11872633318688676

In [124]:
lift = (new_conversion-old_conversion)/old_conversion
lift

-0.001447017918889515

In [125]:
# ab_df['timestamp'] = pd.to_datetime(ab_df['timestamp'], format='%m/%d-%Y hh:mm AM')
ab_df['timestamp'] = pd.to_datetime(ab_df['timestamp'])
# Extract the week from the 'DateColumn' and create a new column 'Week'
ab_df['Week'] = ab_df['timestamp'].dt.isocalendar().week

# Display the DataFrame with the 'Week' column
ab_df

C:\Users\anshulgarg\AppData\Local\Temp\ipykernel_48248\2213152422.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ab_df['timestamp'] = pd.to_datetime(ab_df['timestamp'])


,user_id,timestamp,group,landing_page,converted,Week
0,851104,2017-01-21 22:11:00,control,old_page,0,3
1,804228,2017-01-12 08:01:00,control,old_page,0,2
2,661590,2017-01-11 16:55:00,treatment,new_page,0,2
3,853541,2017-01-08 18:28:00,treatment,new_page,0,1
4,864975,2017-01-21 01:52:00,control,old_page,1,3
...,...,...,...,...,...,...
294473,751197,2017-01-03 22:28:00,control,old_page,0,1
294474,945152,2017-01-12 00:51:00,control,old_page,0,2
294475,734608,2017-01-22 11:45:00,control,old_page,0,3
294476,697314,2017-01-15 01:20:00,control,old_page,0,2


In [126]:
ab_df.groupby('Week').count()

,user_id,timestamp,group,landing_page,converted
Week,,,,,
1,83745,83745,83745,83745,83745
2,91380,91380,91380,91380,91380
3,91056,91056,91056,91056,91056
4,20509,20509,20509,20509,20509


In [127]:
ab_df[(ab_df['group']=='control') & (ab_df['converted']==1)]

,user_id,timestamp,group,landing_page,converted,Week
4,864975,2017-01-21 01:52:00,control,old_page,1,3
15,644214,2017-01-22 02:05:00,control,old_page,1,3
28,913579,2017-01-24 09:11:00,control,old_page,1,4
36,831737,2017-01-11 21:18:00,control,old_page,1,2
43,862225,2017-01-08 14:49:00,control,old_page,1,1
...,...,...,...,...,...,...
294383,728029,2017-01-11 11:17:00,control,old_page,1,2
294385,850065,2017-01-17 11:57:00,control,old_page,1,3
294405,712217,2017-01-11 10:34:00,control,old_page,1,2
294420,795742,2017-01-09 01:06:00,control,old_page,1,2


In [137]:
control_converted = len(ab_df[(ab_df['group']=='control') & (ab_df['converted']==1)])
control_not_converted = len(ab_df[(ab_df['group']=='control') & ~(ab_df['converted']==1)])
treatement_converted = len(ab_df[(ab_df['group']=='treatment') & (ab_df['converted']==1)])
treatement_not_converted = len(ab_df[(ab_df['group']=='treatment') & ~(ab_df['converted']==1)])


In [138]:
np.array([[control_converted,control_not_converted],[treatement_converted,treatement_not_converted]])

array([[ 17220, 126073],
       [ 17025, 126372]])